In [1]:
emotion = "anger"
emotion = "fear"
emotion = "sadness"
emotion = "joy"

In [2]:
import os
import time
import keras
import numpy
import random
import tensorflow

2022-04-03 14:05:10.193209: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-03 14:05:10.193251: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
from keras import *
from keras.utils import *
from keras.models import *
from tensorflow.keras.utils import to_categorical

In [4]:
import re
import csv
from pyspark import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from sklearn.model_selection import KFold

In [5]:
sc = SparkContext("local")
sqlContext = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/03 14:05:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/03 14:05:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
max_features = 20000
sequence_length = 100 #max number of words in one tweet

In [7]:
def str2float(input):
    try:
        return float(input)
    except:
        return None

udf_str2float = udf(str2float, FloatType())

# data conversion functions

## function to convert text file to text and tag list

In [8]:
def convert_file_to_text_and_tag_list(
    emotion_tag,
    data_file,
    ):
    schema = StructType()\
        .add("tweet_id",StringType(),True)\
        .add("text",StringType(),True)\
        .add("emotion",StringType(),True)\
        .add("intensity",FloatType(),True)
    train = sqlContext.read.format('csv')\
        .options(delimiter='\t')\
        .schema(schema)\
        .load(data_file)\
        .withColumn("intensity", udf_str2float("intensity"))
    train.registerTempTable("train")
    train_list = sqlContext.sql(u"""
        SELECT *, 
        CASE 
            WHEN emotion = '%s' THEN 1
            ELSE 0
        END AS label
        FROM train
        WHERE emotion IS NOT NULL
        """%(emotion_tag)).collect()
    texts = [r.text for r in train_list]
    tags = [r.label for r in train_list]
    return texts, tags

## function to convert text file to text and intensentiy score list

In [9]:
def convert_file_to_text_and_score_list(
    emotion_tag,
    data_file,
    ):
    udf_str2float = udf(str2float, FloatType())
    schema = StructType()\
        .add("tweet_id",StringType(),True)\
        .add("text",StringType(),True)\
        .add("emotion",StringType(),True)\
        .add("intensity",FloatType(),True)
    train = sqlContext.read.format('csv')\
        .options(delimiter='\t')\
        .schema(schema)\
        .load(data_file)\
        .withColumn("intensity", udf_str2float("intensity"))
    train.registerTempTable("train")
    train_list = sqlContext.sql(u"""
        SELECT *
        FROM train
        WHERE emotion = '%s' AND intensity IS NOT NULL
        """%(emotion_tag)).collect()
    texts = [r.text for r in train_list]
    scores = [r.intensity for r in train_list]
    return texts, scores

# build the model

# parameters

## convert text list to a input format of deep learning model

In [10]:
def texts_to_input(
    texts,
    sequence_length = 100,
    ):
    word_id_sequence = map(lambda x: 
        keras.preprocessing.text.one_hot(x, n=max_features), 
        texts)
    word_id_sequence = list(word_id_sequence)
    #print(word_id_sequence)
    x = numpy.array(word_id_sequence)
    x = keras.preprocessing.sequence.pad_sequences(
        x, padding="post",
        maxlen=sequence_length,
    )
    return x

texts_to_input(["Never dull moment here"])

## function of building the model of text emotion tag 

In [11]:
embedding_dim = 300
dropout_rate = 0.2
filters = 128

In [12]:
def emotion_tagger_model_building(
    embedding_dim = 300,
    filters = 128,
    kernel_size = 2,
    dropout_rate = 0.2,
    sequence_length = 100,
    ):
    # A integer input for vocab indices.
    inputs = keras.Input(shape=(sequence_length, ))
    # Next, we add a layer to map those vocab indices into a space of dimensionality
    # 'embedding_dim'.
    x = layers.Embedding(max_features, embedding_dim)(inputs)
    x = layers.Dropout(dropout_rate)(x)
    # Conv1D + global max pooling
    x = layers.Conv1D(filters, kernel_size, padding="valid", activation="relu")(x)
    x = layers.Conv1D(filters, kernel_size, padding="valid", activation="relu")(x)
    x = layers.GlobalMaxPooling1D()(x)
    # We add a vanilla hidden layer:
    x = layers.Dense(filters, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(2, 
        activation="softmax",
         name="predictions")(x)
    model = keras.Model(inputs, predictions)
    model.compile(
        loss="categorical_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"])
    return model

## function of train the model of emotion tagger with data

In [13]:
def train_tagger(texts,
    tags,
    tagger_model_path = None,
    tagger_model_weight_path = None,
    tagger_model_json_path = None,
    epochs = 10,
    validation_split = 0.1,
    dropout_rate = 0.2,
    ):
    tagger_model = emotion_tagger_model_building(
        dropout_rate = dropout_rate,
        )
    '''
    prepare the text input

    texts = [
        "i feel so fear",
        "nothing is wrong"
        ]
    '''
    x = texts_to_input(texts)
    '''
    prepare the output
    '''
    y = numpy.array(tags)
    y = to_categorical(y)
    print(x.shape, y.shape)
    print(numpy.sum(y, axis = 0))
    # Fit the model using the train and test datasets.
    tagger_model.fit(x, y, 
        validation_split=validation_split, 
        epochs=epochs)
    # serialize model to JSON
    if tagger_model_json_path is not None:
        model_json = tagger_model.to_json()
        with open(tagger_model_json_path, 'w+') as json_file:
            json_file.write(model_json)
    # serialize weights to HDF5
    if tagger_model_weight_path is not None:
        tagger_model.save_weights(tagger_model_weight_path)
    if tagger_model_path is not None:
        tagger_model.save(tagger_model_path)
    return tagger_model

# function of intensity score model building

In [14]:
def emotion_scorer_model_building(
    embedding_dim = 300,
    filters = 128,
    kernel_size = 2,
    dropout_rate = 0.2,
    sequence_length = 100,
    ):
    # A integer input for vocab indices.
    inputs = keras.Input(shape=(sequence_length, ))
    # Next, we add a layer to map those vocab indices into a space of dimensionality
    # 'embedding_dim'.
    x = layers.Embedding(max_features, embedding_dim)(inputs)
    x = layers.Dropout(dropout_rate)(x)
    # Conv1D + global max pooling
    x = layers.Conv1D(filters, kernel_size, padding="valid", activation="relu")(x)
    x = layers.Conv1D(filters, kernel_size, padding="valid", activation="relu")(x)
    x = layers.GlobalMaxPooling1D()(x)
    # We add a vanilla hidden layer:
    x = layers.Dense(filters, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(1, 
        activation="sigmoid",
        name="predictions")(x)
    model = keras.Model(inputs, predictions)
    model.compile(
        loss="mse", 
        optimizer="adam", 
        metrics=[metrics.mean_absolute_error])
    return model

# function of training intensity score model training

In [15]:
def train_scorer(texts,
    scores,
    scorer_model_path,
    epochs = 10,
    validation_split=0.1,
    ):
    scorer_model = emotion_scorer_model_building()
    '''
    prepare the text input
    texts = [
        "i feel so fear",
        "nothing is wrong"
        ]
    '''
    x = texts_to_input(texts)
    '''
    prepare the output
    '''
    y = numpy.array(scores)
    print(x.shape, y.shape)
    # Fit the model using the train and test datasets.
    scorer_model.fit(x, y, 
        validation_split=0.1, 
        epochs=epochs)
    scorer_model.save(
        scorer_model_path,
        save_format='h5')
    return scorer_model

## training the model

### K-fold cross validation function of tagger

In [16]:
def tagger_model_fold_cross_validation(
    texts,
    tags,
    n_splits = 5,
    ):
    #convert data to numpy arrays
    x = texts_to_input(texts)
    y = numpy.array(tags)
    y = to_categorical(y)
    #make the k folds
    kfold = KFold(n_splits=5, shuffle=True)
    #
    acc_per_fold = []
    loss_per_fold = []
    #
    fold_no = 1
    for train, test in kfold.split(x, y):
        tagger_model = emotion_tagger_model_building()
        tagger_model.fit(
            x[train], y[train], 
            epochs = 5,
            verbose = 0)
        scores = tagger_model.evaluate(
            x[test], y[test], 
            verbose = 0)
        print('accuracy of the {}-th fold:{}'.format(
            fold_no,
            scores[1]))
        acc_per_fold.append(scores[1])
        loss_per_fold.append(scores[0])
        # Increase fold number
        fold_no = fold_no + 1
    ###
    acc_10_fold_cross_validation = numpy.mean(numpy.array(acc_per_fold))
    print('accuracy of {}-fold cross validation:\t{}'.format(
        n_splits,
        acc_10_fold_cross_validation,
        ))

### K-fold cross validation function of scorer

In [17]:
def scorer_model_fold_cross_validation(
    texts,
    scores,
    n_splits = 5,
    ):
    #convert data to numpy arrays
    x = texts_to_input(texts)
    y = numpy.array(scores)
    #make the k folds
    kfold = KFold(n_splits=5, shuffle=True)
    #
    acc_per_fold = []
    loss_per_fold = []
    #
    fold_no = 1
    for train, test in kfold.split(x, y):
        tagger_model = emotion_scorer_model_building()
        tagger_model.fit(
            x[train], y[train], 
            epochs = 5,
            verbose = 0)
        scores = tagger_model.evaluate(
            x[test], y[test], 
            verbose = 0)
        print('mse of the {}-th fold:{}'.format(
            fold_no,
            scores[1]))
        acc_per_fold.append(scores[1])
        loss_per_fold.append(scores[0])
        # Increase fold number
        fold_no = fold_no + 1
    ###
    acc_10_fold_cross_validation = numpy.mean(numpy.array(acc_per_fold))
    print('mse of {}-fold cross validation:\t{}'.format(
        n_splits,
        acc_10_fold_cross_validation,
        ))

# training of each emotion

### tagger training 

#### load data

In [18]:
fear_texts, fear_tags = convert_file_to_text_and_tag_list(
    emotion_tag = emotion,
    data_file = "/*.txt")

/usr/local/lib/python3.9/dist-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


#### 5-fold cross validation

In [19]:
tagger_model_fold_cross_validation(
    fear_texts,
    fear_tags,
    )

/tmp/ipykernel_13110/4256197761.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = numpy.array(word_id_sequence)
2022-04-03 14:05:19.565722: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-03 14:05:19.565765: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-03 14:05:19.565782: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (07b0cb0e5eeb): /proc/driver/nvidia/version does not exist
2022-04-03 14:05:19.565950: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is opti

accuracy of the 1-th fold:0.9482323527336121
accuracy of the 2-th fold:0.9318181872367859
accuracy of the 3-th fold:0.9558081030845642
accuracy of the 4-th fold:0.9280303120613098
accuracy of the 5-th fold:0.9482323527336121
accuracy of 5-fold cross validation:	0.9424242615699768


#### train single model

In [20]:
start_time = time.time()

fear_tagger = train_tagger(
    texts = fear_texts,
    tags = fear_tags,
    tagger_model_path = '/data/emoint/{}_tagger.h5'.format(emotion),
    epochs = 10,
    validation_split = 0.1,
    )

end_time = time.time()

(3960, 100) (3960, 2)
[3058.  902.]
Epoch 1/10


/tmp/ipykernel_13110/4256197761.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = numpy.array(word_id_sequence)


112/112 [==============================] - 5s 40ms/step - loss: 0.5016 - accuracy: 0.7800 - val_loss: 0.6485 - val_accuracy: 0.6768
Epoch 2/10
112/112 [==============================] - 4s 39ms/step - loss: 0.1975 - accuracy: 0.9248 - val_loss: 0.3744 - val_accuracy: 0.8939
Epoch 3/10
112/112 [==============================] - 5s 42ms/step - loss: 0.0476 - accuracy: 0.9823 - val_loss: 0.3563 - val_accuracy: 0.8990
Epoch 4/10
112/112 [==============================] - 4s 39ms/step - loss: 0.0106 - accuracy: 0.9972 - val_loss: 0.5716 - val_accuracy: 0.8712
Epoch 5/10
112/112 [==============================] - 4s 39ms/step - loss: 0.0074 - accuracy: 0.9992 - val_loss: 0.3966 - val_accuracy: 0.8965
Epoch 6/10
112/112 [==============================] - 4s 39ms/step - loss: 0.0025 - accuracy: 0.9992 - val_loss: 0.6424 - val_accuracy: 0.8712
Epoch 7/10
112/112 [==============================] - 4s 39ms/step - loss: 0.0082 - accuracy: 0.9992 - val_loss: 0.4173 - val_accuracy: 0.9015
Epoch 8/10

In [21]:
print('training time of scorer:\t{}'.format(end_time - start_time))

training time of scorer:	45.8455126285553


### scorer training

#### laod data

In [22]:
fear_texts, fear_scores = convert_file_to_text_and_score_list(
    emotion_tag = emotion,
    data_file = "/*.txt")

#### 5-fold cross validation

In [23]:
scorer_model_fold_cross_validation(
    fear_texts,
    fear_scores,
    )

/tmp/ipykernel_13110/4256197761.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = numpy.array(word_id_sequence)


mse of the 1-th fold:0.12508437037467957
mse of the 2-th fold:0.1372532695531845
mse of the 3-th fold:0.13800092041492462
mse of the 4-th fold:0.1397487074136734
mse of the 5-th fold:0.14118005335330963
mse of 5-fold cross validation:	0.13625346422195433


#### train single model

In [24]:
start_time = time.time()

fear_tagger = train_scorer(
    texts = fear_texts,
    scores = fear_scores,
    scorer_model_path = '/data/emoint/{}_scorer.h5'.format(emotion),
    epochs = 10,
    validation_split = 0.1,
    )

end_time = time.time()

(902, 100) (902,)
Epoch 1/10


/tmp/ipykernel_13110/4256197761.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = numpy.array(word_id_sequence)


26/26 [==============================] - 2s 44ms/step - loss: 0.0396 - mean_absolute_error: 0.1632 - val_loss: 0.0653 - val_mean_absolute_error: 0.2180
Epoch 2/10
26/26 [==============================] - 1s 40ms/step - loss: 0.0346 - mean_absolute_error: 0.1521 - val_loss: 0.0547 - val_mean_absolute_error: 0.2000
Epoch 3/10
26/26 [==============================] - 1s 39ms/step - loss: 0.0199 - mean_absolute_error: 0.1103 - val_loss: 0.0506 - val_mean_absolute_error: 0.1723
Epoch 4/10
26/26 [==============================] - 1s 39ms/step - loss: 0.0089 - mean_absolute_error: 0.0740 - val_loss: 0.0462 - val_mean_absolute_error: 0.1674
Epoch 5/10
26/26 [==============================] - 1s 39ms/step - loss: 0.0048 - mean_absolute_error: 0.0550 - val_loss: 0.0439 - val_mean_absolute_error: 0.1656
Epoch 6/10
26/26 [==============================] - 1s 41ms/step - loss: 0.0042 - mean_absolute_error: 0.0504 - val_loss: 0.0435 - val_mean_absolute_error: 0.1650
Epoch 7/10
26/26 [===============

In [25]:
print('training time of scorer:\t{}'.format(end_time - start_time))

training time of scorer:	12.015078783035278
